In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.io import loadmat

In [26]:
data = loadmat('ex4data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [27]:
X = data['X']
y = data['y']

X.shape, y.shape

((5000, 400), (5000, 1))

In [28]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False)
y_onehot = encoder.fit_transform(y)
y_onehot.shape

(5000, 10)

In [29]:
y[0], y_onehot[0, :]

(array([10], dtype=uint8), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]))

---

sigmoid函数

In [30]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

---

前向传播函数

In [31]:
def forward_propagate(X, theta1, theta2):
    m = X.shape[0]

    a1 = np.insert(X, 0, values=np.ones(m), axis=1)
    z2 = a1 * theta1.T
    a2 = np.insert(sigmoid(z2), 0, values=np.ones(m), axis=1)
    z3 = a2 * theta2.T
    h = sigmoid(z3)

    return a1, z2, a2, z3, h

---

代价函数

In [32]:
def cost(params, input_size, hidden_size, num_labels, X, y, lamda):
    m = X.shape[0]

    X = np.matrix(X)
    y = np.matrix(y)

    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
    J = 0

    # 根据公式计算代价函数
    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)
    J = J / m
    J += (float(lamda) / (2 * m)) * (np.sum(np.power(theta1[:, 1:], 2)) + np.sum(np.power(theta2[:, 1:], 2)))

    return J

In [33]:
# 初始化设置
input_size = 400
hidden_size = 25
num_labels = 10
lamda = 1

params = (np.random.random(size=hidden_size * (input_size + 1) + num_labels * (hidden_size + 1)) - 0.5) * 0.25

m = X.shape[0]
X = np.matrix(X)
y = np.matrix(y)

theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))

theta1.shape, theta2.shape

((25, 401), (10, 26))

In [34]:
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
a1.shape, z2.shape, a2.shape, z3.shape, h.shape

((5000, 401), (5000, 25), (5000, 26), (5000, 10), (5000, 10))

In [35]:
cost(params, input_size, hidden_size, num_labels, X, y_onehot, lamda)

6.9225760311495055

---

In [36]:
def sigmoid_gradient(z):
    return np.multiply(sigmoid(z), (1 - sigmoid(z)))

反向传播

In [37]:
def backprop(params, input_size, hidden_size, num_labels, X, y, lamda):
    m = X.shape[0]

    X = np.matrix(X)
    y = np.matrix(y)

    theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
    theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
    a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)

    J = 0
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)

    for i in range(m):
        first_term = np.multiply(-y[i, :], np.log(h[i, :]))
        second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
        J += np.sum(first_term - second_term)

    J = J / m

    # 实现反向传播
    for t in range(m):
        a1t = a1[t, :]
        z2t = z2[t, :]
        a2t = a2[t, :]
        ht = h[t, :]
        yt = y[t, :]

        d3t = ht - yt
        z2t = np.insert(z2t, 0, values=np.ones(1))
        d2t = np.multiply((theta2.T * d3t.T).T, sigmoid_gradient(z2t))
        delta1 = delta1 + (d2t[:, 1:]).T * a1t
        delta2 = delta2 + d3t.T * a2t

    # 加入正则化
    delta1[:, 1:] = delta1[:, 1:] + (theta1[:, 1:] * lamda) / m
    delta2[:, 1:] = delta2[:, 1:] + (theta2[:, 1:] * lamda) / m

    # 将梯度矩阵转换为单个数组
    grad = np.concatenate((np.ravel(delta1), np.ravel(delta2)))

    return J, grad

In [38]:
J, grad = backprop(params, input_size, hidden_size, num_labels, X, y_onehot, lamda)
J, grad.shape

(6.9173444821420995, (10285,))

In [40]:
from scipy.optimize import minimize

fmin = minimize(fun=backprop, x0=params,
                args=(input_size, hidden_size, num_labels, X, y_onehot, lamda), method='TNC', jac=True,
                options={'maxiter': 250})
fmin

     fun: 0.3315633616580405
     jac: array([-6.51603167e-02,  2.30451648e-05,  1.11089213e-05, ...,
       -3.94522065e+00, -3.13994532e-01, -6.14140700e+00])
 message: 'Max. number of function evaluations reached'
    nfev: 251
     nit: 14
  status: 3
 success: False
       x: array([ 0.46371429,  0.11522582,  0.05554461, ...,  0.10924439,
       -4.02453082,  4.28751024])

C:\Users\25110\AppData\Local\Temp/ipykernel_5908/690485297.py:17: RuntimeWarning: divide by zero encountered in log
  second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
C:\Users\25110\AppData\Local\Temp/ipykernel_5908/690485297.py:17: RuntimeWarning: invalid value encountered in multiply
  second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))


     fun: 0.3315633616580405
     jac: array([-6.51603167e-02,  2.30451648e-05,  1.11089213e-05, ...,
       -3.94522065e+00, -3.13994532e-01, -6.14140700e+00])
 message: 'Max. number of function evaluations reached'
    nfev: 251
     nit: 14
  status: 3
 success: False
       x: array([ 0.46371429,  0.11522582,  0.05554461, ...,  0.10924439,
       -4.02453082,  4.28751024])

C:\Users\25110\AppData\Local\Temp/ipykernel_5908/690485297.py:17: RuntimeWarning: divide by zero encountered in log
  second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))
C:\Users\25110\AppData\Local\Temp/ipykernel_5908/690485297.py:17: RuntimeWarning: invalid value encountered in multiply
  second_term = np.multiply((1 - y[i, :]), np.log(1 - h[i, :]))


In [ ]:
X = np.matrix(X)
theta1 = np.matrix(np.reshape(params[:hidden_size * (input_size + 1)], (hidden_size, (input_size + 1))))
theta2 = np.matrix(np.reshape(params[hidden_size * (input_size + 1):], (num_labels, (hidden_size + 1))))
a1, z2, a2, z3, h = forward_propagate(X, theta1, theta2)
y_prep = np.array(np.argmax(h, axis=1) + 1)
y_prep

In [ ]:
correct = [1 if a == b else 0 for (a, b) in zip(y_prep, y)]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print('accuracy = {0}%'.format(accuracy * 100))